# Import Libraries

In [1]:
import requests 
from bs4 import BeautifulSoup 
import csv 

In [2]:
from selenium import webdriver

# Requesting url

In [3]:
path = "C:\Program Files (x86)\chromedriver.exe"

In [4]:
driver = webdriver.Chrome(path)

In [5]:
driver.get("https://www.vitals.com/child-psychiatrists/mi/east-lansing")

In [6]:
soup = BeautifulSoup(driver.page_source, 'html.parser')

## Finding doctor profile links

In [7]:
profile_links_res = []

In [8]:
profile_links = soup.find_all('a', attrs = {'class':''})

In [9]:
p = str(profile_links)
res = [i for i in range(len(p)) if p.startswith('''data-label="view_profile" data-non-webmd="view_profile"''', i)] 
index_of_href = []
for i in res:
    index = p[i:].find("href")
#     print(index+i)
    index_of_href.append(index+i)

In [10]:
index_of_view = []
for i in index_of_href:
    index = p[i:].find("View")
    index_of_view.append(index + i)

In [11]:
for i in range(len(index_of_view)):
    temp = p[index_of_href[i] + 6: index_of_view[i] - 2]
    temp = "www.vitals.com" + temp
    profile_links_res.append(temp)

In [12]:
print(profile_links_res)

['www.vitals.com/doctors/Dr_Alyse_Ley.html', 'www.vitals.com/doctors/Dr_Paul_Quinlan.html', 'www.vitals.com/doctors/Dr_Madhvi_Richards.html']


In [13]:
print(len(profile_links_res))

3


## Finding Doctor names

In [14]:
doctor_names_res = []

In [15]:
dr_names = soup.find_all( attrs = {'class':'name'})

In [16]:
dr_names[:3]

[<span class="name">Dr. Alyse Ley, <span class="degree">DO</span></span>,
 <span class="name">Dr. Paul Quinlan, <span class="degree">DO</span></span>,
 <span class="name">Dr. Madhvi Richards, <span class="degree">MD</span></span>]

In [17]:
p = str(dr_names)
index_of_Dr = [i for i in range(len(p)) if p.startswith('''Dr''', i)]
# for i in res:
#     index = p[i:].find("Dr.")
#     index_of_Dr.append(index+i)

In [18]:
p

'[<span class="name">Dr. Alyse Ley, <span class="degree">DO</span></span>, <span class="name">Dr. Paul Quinlan, <span class="degree">DO</span></span>, <span class="name">Dr. Madhvi Richards, <span class="degree">MD</span></span>]'

In [19]:
index_of_comma = []
for i in index_of_Dr:
    index = p[i:].find(",")
    index_of_comma.append(index + i)

In [20]:
for i in range(len(index_of_view)):
    temp = p[index_of_Dr[i]: index_of_comma[i]]
    doctor_names_res.append(temp)

In [21]:
print(doctor_names_res)

['Dr. Alyse Ley', 'Dr. Paul Quinlan', 'Dr. Madhvi Richards']


In [22]:
print(len(doctor_names_res))

3


## Finding Doctor Specialty

In [23]:
doctor_specialty_res = []

In [24]:
specialty = soup.find_all( attrs = {'class':'specialty uppercase'})

In [25]:
p = str(specialty)
index_of_end_bracket = [i for i in range(len(p)) if p.startswith('''>''', i)]

In [26]:
index_of_start_bracket = []
for i in index_of_end_bracket:
    index = p[i:].find("<")
    index_of_start_bracket.append(index + i)

In [27]:
flag = 0
for i in range(len(index_of_view)*2):
    temp = p[index_of_end_bracket[i]+2: index_of_start_bracket[i]].strip()
#     doctor_specialty_res.append(temp)
    if flag == 0:
        doctor_specialty_res.append(temp)
        flag = 1
    else:
        flag = 0

In [28]:
print(doctor_specialty_res)

['Psychiatry, Neurology, Child Psychiatry', 'Adolescent Medicine, Psychiatry, Child Psychiatry', 'Psychiatry, Child Psychiatry']


In [29]:
print(len(doctor_specialty_res))

3


## Finding doctor ratings

In [30]:
ratings = []
number_of_ratings = []

In [31]:
rating = soup.find_all('div', attrs = {'class':'card-rating'})

In [32]:
for i in rating:
    print(i, end = "\n\n\n")

<div class="card-rating">
<aside class="rating-4 count">
<span class="overall">3.9</span>
<div class="stars">
<i class="icons-star grey-text text-lighten-2"></i><i class="icons-star grey-text text-lighten-2"></i><i class="icons-star grey-text text-lighten-2"></i><i class="icons-star grey-text text-lighten-2"></i><i class="icons-star grey-text text-lighten-2"></i> <div style="width: 78%">
<i class="icons-star rating-text"></i><i class="icons-star rating-text"></i><i class="icons-star rating-text"></i><i class="icons-star rating-text"></i><i class="icons-star rating-text"></i> </div>
</div>
</aside>
<a data-non-webmd="rating" href="/doctors/Dr_Alyse_Ley/reviews">(11)</a>
</div>


<div class="card-rating">
<aside class="rating-4 count">
<span class="overall">4.4</span>
<div class="stars">
<i class="icons-star grey-text text-lighten-2"></i><i class="icons-star grey-text text-lighten-2"></i><i class="icons-star grey-text text-lighten-2"></i><i class="icons-star grey-text text-lighten-2"></i

In [33]:
for i in rating:
    if len(ratings) == len(doctor_specialty_res):
        break

    if str(i).find("card-unrated") != -1:
        ratings.append(None)
        number_of_ratings.append(0)
    else:
        index = str(i).find('''class="overall"''')
        index += 16
        index2 = str(i)[index:].find('''</span>''')
        index2 = index2 + index
#         print(str(i)[index:index2])
#         print(len(str(i)[index:index2]))
        ratings.append(float(str(i)[index:index2]))
    
        index = str(i).find('''data-non-webmd="rating"''')
        index2 = str(i)[index:].find('''</a>''')
        index2 += index
        
        index3 = str(i)[index:].find('''">''')
        index3 += index
        number_of_ratings.append(int(str(i)[index3 + 3 : index2-1]))

In [34]:
print(ratings)

[3.9, 4.4, 3.0]


In [35]:
print(number_of_ratings)

[11, 5, 2]


In [36]:
print(len(ratings))

3


In [37]:
print(len(number_of_ratings))

3


In [38]:
# driver.quit()